In [0]:
!pip install -q gpt_2_simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files
import json
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import numpy as np
import random

In [8]:
gpt2.mount_gdrive()
gpt2.copy_checkpoint_from_gdrive()
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


FileNotFoundError: ignored

In [0]:
def sample_random(prefix, temp=0.7):
  return gpt2.generate(sess, length=len(prefix), temperature=temp, prefix=prefix)
def sample_sentiment(prefix, sentiment_vec, max_iter=100, thresh=0):
  """ 
  Sentiment based rejection sampling
  returns the best sample
  Sample either hundred times or when we do better than threshhold distance
  note thresh is between [0,2]. It's an early cutoff
  """
  best_score = 2
  best_result = None
  for i in range(max_iter):
    sample = sample_random(prefix, random.random())
    ss = sid.polarity_scores(sample)
    curr_sent = np.array([ss["compound"], ss["neg"], ss["neu"], ss["pos"]])
    dist = np.linalg.norm(sentiment_vec-curr_sent)
    if (dist < thresh):
      return sample
    if dist < best_score:
      best_result = sample
      best_score = dist
  return best_result

In [0]:
def test_single(sess, checkpoint, sid, train_file, test_instance, save_dest):
  """
  Takes the single testing JSON file as a dict.
  Runs sampling on all 3, saves results to dest dir, each training example to
  its own file
  """
  gpt2.finetune(sess,
                dataset=train_file,
                steps=20,
                restore_from=checkpoint,
                print_every=30,
                sample_every=30,
                save_every=30
                ) # don't want to save
  results = []
  for i in range(1,4):
    inp = test_instance["test{}_input".format(i)]
    tru = test_instance["test{}_true".format(i)]
    id_ = test_instance["test{}_id".format(i)]
    # Generate random sample
    samp_rand = sample_random(inp)
    # Generate sentiment sample
    ss = sid.polarity_scores(sentence)
    sentiment_vec = np.array([ss["compound"], ss["neg"], ss["neu"], ss["pos"]])
    samp_sent = sample_sentiment(inp, sentiment_vec)
    sample = {"id": id_,
              "prefix": inp,
              "truth": tru,
              "random": samp_rand,
              "sentiment": samp_sent
             }
    results.append(sample)
  with open(save_dest, "w") as f:
    json.dump(results, f)   

def run_tests(test_dir, train_dir, sess, checkpoint):
  # Load model from checkpoint
  for f in listdir(train_dir):
    fname = f[:-3]
    train = join(train_dir, f)
    test = join(test_dir, fname+".json")
    if isfile(src) and isfile(test):
      with open(test, "r") as test_file:
        test_instance = json.loads(test_file)[0]
        test_single(sess, checkpoint, train, test_instance, "/content/results/{}_sample.json".format(f))